In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
df1 = pd.read_html(link, header=0)[3]
df2 = pd.read_html(link, header=0)[4]
df3 = pd.read_html(link, header=0)[5]
df4 = pd.read_html(link, header=0)[6]

In [4]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [5]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3.0,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10.0,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10.0,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10.0,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
4,JANUARY,10.0,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
...,...,...,...,...,...,...
190,DECEMBER,18.0,The Father,Sony Pictures Classics / Film4,Florian Zeller (director/screenplay); Christop...,NaN
191,DECEMBER,25.0,Wonder Woman 1984,Warner Bros. Pictures / DC Films / Atlas Enter...,Patty Jenkins (director/screenplay); Geoff Joh...,[184]
192,DECEMBER,25.0,News of the World,Universal Pictures,Paul Greengrass (director/screenplay); Luke Da...,[185]
193,DECEMBER,NaN,NaN,NaN,NaN,NaN


In [6]:
df_2020 = df[['Title','Cast and crew']]

In [7]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...
...,...,...
190,The Father,Florian Zeller (director/screenplay); Christop...
191,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...
192,News of the World,Paul Greengrass (director/screenplay); Luke Da...
193,NaN,NaN


In [8]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '3b293cbccc5e0330718ef7ca99ed0998'

In [9]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [10]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

C:\Users\shaan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery
...,...,...,...
190,The Father,Florian Zeller (director/screenplay); Christop...,Drama
191,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...,Fantasy Action Adventure
192,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western
193,NaN,NaN,None


In [12]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [13]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\shaan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [15]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\shaan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [17]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [18]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [19]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [20]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer
...,...,...,...,...,...,...,...
190,The Father,Florian Zeller (director/screenplay); Christop...,Drama,Florian Zeller,Anthony Hopkins,Olivia Colman,Mark Gatiss
191,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...,Fantasy Action Adventure,Patty Jenkins,Gal Gadot,Chris Pine,Kristen Wiig
192,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western,Paul Greengrass,Tom Hanks,Helena Zengel,Neil Sandilands
193,NaN,NaN,None,nan,nan,NaN,NaN


In [21]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [22]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [23]:
for i in new_df20.columns:
    new_df20[i] = new_df20[i].str.lower()

In [24]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,nicolas pesce,andrea riseborough,demián bichir,john cho,horror mystery,the grudge
1,william eubank,kristen stewart,vincent cassel,jessica henwick,action horror science fiction thriller,underwater
2,miguel arteta,tiffany haddish,rose byrne,salma hayek,comedy,like a boss
3,anthony jerjen,josh hartnett,margarita levieva,chandler riggs,drama thriller crime,inherit the viper
4,andrew desmond,freya tingley,simon abkarian,rutger hauer,horror thriller mystery,the sonata
...,...,...,...,...,...,...
190,florian zeller,anthony hopkins,olivia colman,mark gatiss,drama,the father
191,patty jenkins,gal gadot,chris pine,kristen wiig,fantasy action adventure,wonder woman 1984
192,paul greengrass,tom hanks,helena zengel,neil sandilands,drama western,news of the world
193,nan,nan,NaN,NaN,None,NaN


In [25]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [27]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      4
actor_3_name     18
genres            3
movie_title       2
comb             19
dtype: int64

In [28]:
new_df20 = new_df20.dropna(how='any')

In [29]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [30]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,nicolas pesce,andrea riseborough,demián bichir,john cho,horror mystery,the grudge,andrea riseborough demián bichir john cho nico...
1,william eubank,kristen stewart,vincent cassel,jessica henwick,action horror science fiction thriller,underwater,kristen stewart vincent cassel jessica henwick...
2,miguel arteta,tiffany haddish,rose byrne,salma hayek,comedy,like a boss,tiffany haddish rose byrne salma hayek miguel ...
3,anthony jerjen,josh hartnett,margarita levieva,chandler riggs,drama thriller crime,inherit the viper,josh hartnett margarita levieva chandler riggs...
4,andrew desmond,freya tingley,simon abkarian,rutger hauer,horror thriller mystery,the sonata,freya tingley simon abkarian rutger hauer andr...
...,...,...,...,...,...,...,...
188,kenneth branagh,kenneth branagh,tom bateman,annette bening,crime mystery drama thriller,death on the nile,kenneth branagh tom bateman annette bening ken...
189,craig brewer,eddie murphy,jermaine fowler,arsenio hall,comedy,coming 2 america,eddie murphy jermaine fowler arsenio hall crai...
190,florian zeller,anthony hopkins,olivia colman,mark gatiss,drama,the father,anthony hopkins olivia colman mark gatiss flor...
191,patty jenkins,gal gadot,chris pine,kristen wiig,fantasy action adventure,wonder woman 1984,gal gadot chris pine kristen wiig patty jenkin...


In [31]:
old_df = pd.read_csv('final_data.csv')

In [32]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,james cameron,cch pounder,joel david moore,wes studi,action adventure fantasy sci-fi,avatar,cch pounder joel david moore wes studi james c...
1,gore verbinski,johnny depp,orlando bloom,jack davenport,action adventure fantasy,pirates of the caribbean: at world's end,johnny depp orlando bloom jack davenport gore ...
2,sam mendes,christoph waltz,rory kinnear,stephanie sigman,action adventure thriller,spectre,christoph waltz rory kinnear stephanie sigman ...
3,christopher nolan,tom hardy,christian bale,joseph gordon-levitt,action thriller,the dark knight rises,tom hardy christian bale joseph gordon-levitt ...
4,doug walker,doug walker,rob walker,unknown_actor,documentary,star wars: episode vii - the force awakens ...,doug walker rob walker unknown_actor doug walk...
...,...,...,...,...,...,...,...
5939,greta gerwig,saoirse ronan,emma watson,florence pugh,drama romance,little women,saoirse ronan emma watson florence pugh greta ...
5940,sam mendes,george mackay,dean-charles chapman,mark strong,war drama action history,1917,george mackay dean-charles chapman mark strong...
5941,destin daniel cretton,michael b. jordan,jamie foxx,brie larson,drama crime,just mercy,michael b. jordan jamie foxx brie larson desti...
5942,chinonye chukwu,alfre woodard,wendell pierce,aldis hodge,drama,clemency,alfre woodard wendell pierce aldis hodge chino...


In [33]:
final_df = old_df.append(new_df20,ignore_index=True)

In [34]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,james cameron,cch pounder,joel david moore,wes studi,action adventure fantasy sci-fi,avatar,cch pounder joel david moore wes studi james c...
1,gore verbinski,johnny depp,orlando bloom,jack davenport,action adventure fantasy,pirates of the caribbean: at world's end,johnny depp orlando bloom jack davenport gore ...
2,sam mendes,christoph waltz,rory kinnear,stephanie sigman,action adventure thriller,spectre,christoph waltz rory kinnear stephanie sigman ...
3,christopher nolan,tom hardy,christian bale,joseph gordon-levitt,action thriller,the dark knight rises,tom hardy christian bale joseph gordon-levitt ...
4,doug walker,doug walker,rob walker,unknown_actor,documentary,star wars: episode vii - the force awakens ...,doug walker rob walker unknown_actor doug walk...
...,...,...,...,...,...,...,...
6115,kenneth branagh,kenneth branagh,tom bateman,annette bening,crime mystery drama thriller,death on the nile,kenneth branagh tom bateman annette bening ken...
6116,craig brewer,eddie murphy,jermaine fowler,arsenio hall,comedy,coming 2 america,eddie murphy jermaine fowler arsenio hall crai...
6117,florian zeller,anthony hopkins,olivia colman,mark gatiss,drama,the father,anthony hopkins olivia colman mark gatiss flor...
6118,patty jenkins,gal gadot,chris pine,kristen wiig,fantasy action adventure,wonder woman 1984,gal gadot chris pine kristen wiig patty jenkin...


In [35]:
final_df.to_csv('main_data.csv',index=False)